In [1]:
import requests
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

APIKEY = os.getenv('API_KEY')

In [6]:
def fetch_data(crypto_id, currency='usd', days='90', apikey=APIKEY):
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart?vs_currency={currency}&days={days}"

    headers = {
        "accept": "application/json",
        "x_cg_demo_api_key": apikey
    }

    response = requests.get(url, headers=headers)
    data = response.json()
    prices = [price[1] for price in data['prices']]

    return prices

In [7]:
fetch_data("bitcoin")

[62479.231973519934,
 62240.6083876244,
 62384.34904441377,
 62560.61441012541,
 62514.66233700978,
 62604.096795218546,
 62504.51859636706,
 62770.26425664377,
 62766.14682170429,
 62152.158564241465,
 62525.15635385936,
 62331.602816503626,
 61963.1594430075,
 62121.59452794766,
 62325.37670520698,
 62373.092674608226,
 62105.27060195742,
 62185.230423727575,
 62115.919752113165,
 62299.981991661436,
 62272.6586800959,
 62334.18871429603,
 62307.406121183434,
 62490.39392748502,
 62301.0120009861,
 62228.21482411327,
 62199.03458257609,
 62222.525571365324,
 62135.36928631705,
 62144.517600515544,
 61979.063023858784,
 61843.712560203414,
 61935.669410618335,
 62273.12429552298,
 61752.45976427412,
 61786.15413301724,
 61117.826773189954,
 61045.573897756294,
 60486.87709708697,
 60743.1844472768,
 60648.86752390266,
 60583.12205498274,
 60388.16814788513,
 60749.228818156014,
 60807.96354622658,
 60806.83687955058,
 60823.19050349816,
 60868.330109499504,
 61039.07191204449,
 60816.